In [1]:
import numpy as np

class PIDController():

    AUTO_MODE = 1
    OFF_MODE = 0

    def __init__(self, pgain=1, itime=1, dtime=0, auto_max=100,
                 auto_min=0, do_ifactor=False, anti_windup_backcalc=True,
                 beta=1, linearity=1, alpha=1, deadband=0):

        self.auto_min = auto_min
        self.auto_max = auto_max
        self.pgain = pgain
        self.itime = itime
        if self.itime:
            self.oneoveritime = 1 / self.itime
        else:
            self.oneoveritime = 0
        self.dtime = dtime
        self.dif = do_ifactor
        self.awb = anti_windup_backcalc

        self.accumulated_error = 0
        self.bump = 0
        self.last_output = 0
        self.last_error = 0
        self.last_pv = 0
        self.b = beta
        self.l = linearity
        self.a = alpha
        self.deadband = deadband

    def off_to_auto(self, pv, sp):
        """
        Calculate bump for off-to-auto transfer
        :param pv: current process temp to use for bumpless xfer calculation
        """
        self.man_to_auto(pv, sp, 0)

    def man_to_auto(self, pv, sp, op):
        #err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
        err_for_pgain = (sp-pv) * self.b
        uk0 = self.pgain * err_for_pgain
        self.bump = op - uk0
        self.last_pv = pv
        
    def auto_to_auto(self, pv, op):
        self.man_to_auto(pv, pv, op)
        
    set_bump = man_to_auto

    def reset(self):
        self.accumulated_error = 0
        self.last_pv = 0
        self.last_error = 0        
    
    def step(self, pv, sp, dt=1):
        err = sp - pv
#         if err >= self.deadband:
#             err -= self.deadband
#         elif err <= -self.deadband:
#             err += self.deadband
#         else:
#             err = 0
#         err_for_pgain = self.b*err*(self.l+(1-self.l)*(abs(self.b*err))/100)
        
        dpv = pv - self.last_pv
        ierr = (err + self.last_error) / 2 * dt
        
        # SP_Range is taken to be 100 for both of the below lines
        # since range is calculated as +/- middle (?)
        # so -100-100 is actually range of 100 according to LV (??)
        err_for_pgain = (self.b*sp-pv)*(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
#         err_for_pgain = err * self.b
        if self.dif:
            ierr *= 1 / (1 + err * err * 10 / (100*100))  # Labview's ifactor thingy
        
        # bump (aka controller bias) isn't normally
        # included in Up, but no one else reads my 
        # code :)
        Up = self.bump + self.pgain * err_for_pgain
        Ui = self.oneoveritime * (ierr + self.accumulated_error) * self.pgain
        Ud = self.dtime * dpv * self.pgain * self.a
        Uk = Up + Ui + Ud
        
        # XXX debugging
        self.Ui = Ui
        self.Uk = Uk
        self.Ud = Ud
        self.Up = Up
        
        # Coercion & back calculation      
        back_calc = False
        if Uk > self.auto_max:
            Uk = self.auto_max
            back_calc = True
            am = self.auto_max
        elif Uk < self.auto_min:
            Uk = self.auto_min
            back_calc = True
            am = self.auto_min
            
        if back_calc and self.awb:
            ierr = self.itime / self.pgain * (am - Up - Ud) - self.accumulated_error
            
        self.accumulated_error += ierr
        self.last_output = Uk
        self.last_pv = pv
        self.last_error = err
        self.last_ierr = ierr
        
        return Uk

    def __repr__(self):
        return "Output: %.2f Pgain: %.1f Itime: %.2f AccumError: %.4f" % (self.last_output,
                                                                          self.pgain,
                                                                          self.itime,
                                                                          self.accumulated_error)
    __str__ = __repr__
    

In [2]:
def assert_equal(a,b):
    assert a == b, (a, b)
def test_trap_integration1():
    data = [
        (0, 1, .5),
        (0, 2, 2),
        (0, 3, 4.5)
    ]
    p = PIDController(1,1)
    for pv, sp, op in data:
        p.off_to_auto(pv, sp)
        res_op = p.step(pv, sp)
        assert_equal(p.last_error, sp)
        assert_equal(p.accumulated_error, op)
        assert_equal(res_op, op)
test_trap_integration1()

def check_Ui_backcalc(it, pg, Up, Ud, ierr, accum, val):
    Ui2 = it * (ierr + accum) * pg
    Uk2 = Up + Ud + Ui
    Uk2 = round(Uk2, 8)
    am = round(val, 8)
    assert Uk2 == am, (Up, Ud, Ui, Uk2, val)

In [3]:
def m2s(m):
    return m*60
def s2m(s):
    return s/60
def h2s(h):
    return m2s(h*60)

from types import FunctionType, MethodType
def printdir(o):
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o, a)
            if not isinstance(v, (FunctionType, MethodType)):
                print(a, v)

from collections import deque
class DelayBuffer(deque):
    def __init__(self, delay=30, startvalue=0):
        delay = int(delay)
        self.delay = delay
        super().__init__(startvalue for _ in range(delay + 1))

    def cycle(self, hd):
        self[0] = hd
        self.rotate(1)
        return self[0]
    
class DelaySink():
    def __init__(self, delay, initial, pc_at_delay=0.95, dt=1):
        assert 0 < pc_at_delay <= 1, "don't be a retard"
        if delay:
            self.df = 1 - (1-pc_at_delay)**(dt/delay)
            self.sink = initial / self.df  - initial # fill the sink
        else:
            self.df = 0
            self.sink = 0
        self.dt = dt
        if self.sink < 0:
            self.sink = 0
        
    def delay(self, op):
        self.sink += op * self.dt
        dNdt = self.sink*self.df
        self.sink -= dNdt
        return dNdt

In [4]:
TOTAL_RVOLUME = {
    3: 4,
    15: 17,
    80: 90,
}

class DOProcess():
    """ DO Process
    Uses cascaded calculation model,
    First using HeadspaceProcess to estimate
    headspace gas concentrations, and using
    the result to calculate change in PV. 
    """
    # process gain constant units of  
    # % per delta% per hour 
    # dpv/dt = k*([O2]/0.2095 * 100 - DOPV)
    # where [02] is volume fraction, DOPV is in %
    default_k = 0.1306
    
    # consumption rate constant in units of % per hour
    # presumably, consumption rate is "Constant"
    # within approximation, as long as cellular conditions
    # don't transition between aerobic and anaerobic
    default_c = 0
    
    # volume fraction of CO2, N2, O2 in air
    # note the remaining 1% is trace gasses
    AIR_CNO = (0.0004, 0.7809, 0.2095)
    
    def __init__(self, main_gas=1, initial_pv=60, initial_cno=AIR_CNO, reactor_size=80, volume=55, delay=0):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial_pv).cycle
        self.k = self.default_k / 3600  # in seconds
        self.c = self.default_c / 3600  # in seconds
        self._reactor_size = reactor_size
        self._volume = volume
        self.main_gas = main_gas
    
        hs_volume = TOTAL_RVOLUME[reactor_size] - volume
        c, n, o = initial_cno
        self.hp = HeadspaceProcess(hs_volume, c, n, o)
        
    @property
    def volume(self): 
        return self._volume
    
    def set_values(self, k=None, c=None):
        if k is not None:
            self.k = k / 3600
        if c is not None:
            self.c = c / 3600
    
    @volume.setter
    def volume(self, v):
        self._volume = v
        self.hp.vol = TOTAL_RVOLUME[self._reactor_size] - v

    def step(self, pv, co2_req, n2_req, o2_req, air_req):
        self.hp.calc_gas(self.main_gas, co2_req, n2_req, o2_req, air_req)
        o2 = self.hp.o2A
        pvarg = (o2 * 100 / 0.2095 - pv)
        dpv = self.k * pvarg
        pv += dpv
        pv -= self.c
        if pv < 0:
            return 0
        return pv      
    

class HeadspaceProcess():
    def __init__(self, vol, co2A=0, n2A=0.78, o2A=0.21):
        """ 
        vol: volume headspace (L)
        main_gas: main gas flow rate (L/min)
        co2: CO2 request (%)
        n2: N2 request (%)
        
        Calculate headspace concentration. DT = 1 second. 
        """
        self.vol = vol
        self.co2A = co2A
        self.n2A = n2A
        self.o2A = o2A
        self.airA = 1 - (co2A+n2A+o2A)
        
    def step(self, main_gas, co2_req, n2_req, o2_req, air_req):
        self.calc_gas(main_gas, co2_req, n2_req, o2_req, air_req)
        return self.o2A
    
    def calc_gas(self, main_gas, co2_req, n2_req, o2_req, air_req):
        """ 
        Calculate new headspace gas concentration.
        Assumptions:
            - Mixing time is 0 seconds, i.e. gas mixture is always perfectly mixed
            - Gas lost from headspace is always equal to gas entering headspace
            - Concentration of individual gasses lost match headspace concentration of gasses. 
            - Mass transfer coefficient of Air <-> liquid k <<< Mg (no volume lost due to absorption to liquid)
            - Air O2: 20.95%, N2: 78.09%, CO2=0.04%, other = 1-(o2+n2+co2)
        """
        mg = main_gas / 60
        
        # volume of gas lost
        co2L = mg * self.co2A
        n2L = mg * self.n2A
        o2L = mg * self.o2A
        airL = mg * self.airA
        
        # volume of gas gained
        # airG is non-corrected volume of air added
        co2G = mg * co2_req
        n2G = mg * n2_req
        o2G = mg * o2_req
        airG = mg * air_req
        
        # air contribution to gases
        # airG is corrected to reflect
        # the volume of non-tracked gases
        # ie the 1% of trace elements or so
        co2G += 0.0004 * airG
        o2G += 0.2095 * airG
        n2G += 0.7809 * airG
        airG = .0092 * airG
        
        # new volume of each gas
        co2V = co2G - co2L + self.co2A * self.vol
        n2V = n2G - n2L + self.n2A * self.vol
        o2V = o2G - o2L + self.o2A * self.vol
        airV = airG - airL + self.airA * self.vol
        
        # finally, new percentages
        self.co2A = co2V / self.vol
        self.n2A = n2V / self.vol
        self.o2A = o2V / self.vol
        self.airA = airV / self.vol
        
        # Sanity checks
        # Errors should be on the order of 
        # floating point rounding error ~1e-14
        self._err_ = self.vol - sum((co2V, n2V, o2V, airV))
        self._err2_ = 1 - (self.co2A + self.n2A + self.o2A + self.airA)
        self._err3_ = (co2L+n2L+o2L+airL) - (co2G + n2G + o2G + airG)
            

In [5]:
class GasController():
    def __init__(self, co2mfcmax, n2mfcmax, o2mfcmax, airmfcmax):
        self.cm = co2mfcmax
        self.nm = n2mfcmax
        self.om = o2mfcmax
        self.am = airmfcmax
   
    def request(self, main_gas, co2_req, n2_req, o2_req):

        co2max = self.cm / main_gas
        co2pc = co2max if co2_req > co2max else co2_req

        o2_req_max = 1 - co2pc
        if o2_req > o2_req_max: o2_req = o2_req_max
        o2max = self.om / main_gas
        o2pc = o2max if o2_req > o2max else o2_req
        
        n2_req_max = 1 - co2pc - o2pc
        if n2_req > n2_req_max: n2_req = n2_req_max
        n2max = self.nm / main_gas
        n2pc = n2max if n2_req > n2max else n2_req
        
        apc = 1 - co2pc - o2pc - n2pc
        
        return co2pc, n2pc, o2pc, apc

In [6]:
def _round(dig, *args):
    rv = []
    for a in args:
        rv.append(round(a, dig))
    return rv
def test_gas_controller():
    ctrl = GasController(1, 10, 2, 10)
    
    c,o,n = (0.8, 0.3, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(0.5, c,n,o))
    assert rc == 0.8, rc
    assert ro == 0.2, ro
    assert rn == 0, rn
    assert ra == 0, ra
    
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0.2, ro
    assert rn == 0.7, rn
    assert ra == 0, ra
    
    c,o,n = (0.8, 0, 1)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.1, rc
    assert ro == 0, ro
    assert rn == 0.9, rn
    assert ra == 0, ra
    
    c,o,n = (0.07, 0.1, 0.3)
    rc, rn, ro, ra = _round(5, *ctrl.request(10, c,n,o))
    assert rc == 0.07, rc
    assert ro == 0.1, ro
    assert rn == 0.3, rn
    assert ra == 0.53, ra
    
    for mg in (0.5, 1, 5, 10):
        for c in 0.1, 0.5, 0.8, 1:
            for o in 0.1, 0.5, 0.8, 1:
                for n in 0.1, 0.5, 0.8, 1:
                    args = (mg, c, n, o)
                    ec = round(min(1/mg, c), 5)
                    eo = round(min(2/mg, (1-ec), o),5)
                    en = round(min(1-ec-eo, n),5)
                    ea = round(1-ec-eo-en,5)
                    rc, rn, ro, ra = _round(5, *ctrl.request(*args))
                    assert rc == ec, (rc, ec, args)
                    assert ro == eo, (ro, eo, args, (rc, rn, ro, ra))
                    assert rn == en, (rn, en, args)
                    assert ra >= 0 and ra == ea, (ra, ea, args)
    
test_gas_controller()

In [7]:
ypv = []
yco2 = []
yn2 = []
yo2 = []
err = []
err2 = []
err3 = []
pv = 60
mg = 1
co2r = 0.07
n2r = 0.10
o2r = 0.20
proc = DOProcess(mg, pv, DOProcess.AIR_CNO, 80, 55)
seconds = 72000
for _ in range(seconds):
    pv = proc.step(pv, co2r, n2r, o2r)
    ypv.append(pv)
    yco2.append(proc.hp.co2A)
    yn2.append(proc.hp.n2A)
    yo2.append(proc.hp.o2A)
    err.append(proc.hp._err_)
    err2.append(proc.hp._err2_)
    err3.append(proc.hp._err3_)
    
for _ in range(seconds):
    pv = proc.step(pv, 0.14, 0.3, 0.05)
    ypv.append(pv)
    yco2.append(proc.hp.co2A)
    yn2.append(proc.hp.n2A)
    yo2.append(proc.hp.o2A)
    err.append(proc.hp._err_)
    err2.append(proc.hp._err2_)
    err3.append(proc.hp._err3_)
    
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
fm = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))
plt.close()
x = np.arange(len(ypv)) / 3600


def lowpass(data, alpha):
    y = np.zeros_like(data)
    ys = data[0]
    for i, d in enumerate(data):
        ys += alpha * (d - ys)
        y[i] = ys
    return y
# err = lowpass(err, 1/1000)
# err2 = lowpass(err2, 1/1000)
# ax1 = plt.gca()
# plt.plot(x, err, "blue", ls="-", label="err")
# plt.plot(x, err2*100, "green", ls="-", label="err2")
# plt.gca().twinx().plot(x, err3, "purple", ls="-", label="PV")

plt.plot(x, ypv, "blue", ls="-", label="PV")
ax2 = plt.gca().twinx()
ax2.plot(x, yco2, "purple", ls="-", label="CO2")
ax2.plot(x, yo2, "red", ls="-", label="O2")
ax2.plot(x, yn2, "green", ls="-", label="N2")


plt.legend(loc="center left")
plt.gca().yaxis.set_major_formatter(fm)

air_req = 1 - (o2r+n2r+co2r)
o2 = o2r + air_req * 0.2095
n2 = n2r + air_req * 0.7809
co2 = co2r + air_req * 0.0004
# ax2.axhline(y=o2, ls="--", color="black")
# ax2.axhline(y=n2, ls="--", color="black")
# ax2.axhline(y=co2, ls="--", color="black")
# plt.axvline(x=seconds/3600, ls="--", color="black")
plt.show()

TypeError: step() missing 1 required positional argument: 'air_req'

In [13]:
import json

class OptionCategory():
    def __iter__(self):
        d = self.__class__.__dict__.copy()
        d.update(self.__dict__)
        ob = ((k, v) for k,v in d.items() if k[0] != "_" and k[-1]!="_")
        return iter(ob)
    
    def __repr__(self):
        buf = []
        for k, v in self:
            if isinstance(v, OptionCategory):
                v = "<More Options...>"
            buf.append("%s: %s" %(k,v))
        return "\n".join(buf)
    
    def __init__(self):
        self.__dict__.update(self.__class__.__dict__)
    
    def __setattr__(self, k, v):
        if k not in self.__dict__:
            raise AttributeError(k)
        object.__setattr__(self, k, v)
        
    def asdict(self):
        basedict = dict(iter(self))
        rv = {}
        for k in basedict:
            v = basedict[k]
            if isinstance(v, OptionCategory):
                v = v.asdict()
            rv[k] = v
        return rv
    
    def jsonify(self, **kw):
        kw['indent'] = kw.get('indent') or 4
        return json.dumps(self.asdict(), **kw)

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    deadband = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72

class SimOps(OptionCategory):
    
    o2_pid = PIDOps()
    o2_pid.p = 5
    o2_pid.i = 5
    o2_pid.d = 0
    o2_pid.amax = 100
    o2_pid.amin = 0
    o2_pid.beta = 1
    o2_pid.linearity = 1
    o2_pid.alpha = 1
    
    n2_pid = PIDOps()
    n2_pid.p = 5
    n2_pid.i = 5
    n2_pid.d = 0
    n2_pid.amax = 0
    n2_pid.amin = 0
    n2_pid.beta = 1
    n2_pid.linearity = 1
    n2_pid.alpha = 1
    
    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 2
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    
    delay = 0
    end = 10000
    initial_actual_cno = DOProcess.AIR_CNO
    initial_request_cno = (0.07, 0, 0)
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    mode = "m2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = reactor_size * 55/80


In [14]:

def do_sim(ops):
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp-db, o2_req*100)
        n2_pid.man_to_auto(pv, sp+db, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(ops.k*ops.k_mult, ops.c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    end = ops.end
    
    data = [(t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a)]
    data2 = [(0, 0, 0)]
    while True:
        t += 1
        o2_req = o2_pid.step(pv, sp-db) / 100
        n2_req = n2_pid.step(pv, sp+db) / 100
        co2_req, o2_req, n2_req, air_req = ctrl.request(mg, co2_req, o2_req, n2_req)
        pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)
        co2a = proc.hp.co2A
        n2a = proc.hp.n2A
        o2a = proc.hp.o2A
        data.append((t, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a))
        data2.append((n2_pid.Uk, n2_pid.Up, n2_pid.Ui))
        if t >= end:
            break
    return data, data2

In [15]:
ops = SimOps()
ops.delay = 0
ops.end = 10000
ops.initial_actual_cno = DOProcess.AIR_CNO
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 100
ops.set_point = 90
ops.k = DOProcess.default_k
ops.c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55

In [16]:
# O2 pid
ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.d = 0
ops.o2_pid.alpha = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.linearity = 1
ops.o2_pid.beta = 1

In [17]:
# N2 PID
ops.n2_pid.p = -5
ops.n2_pid.i = 5
ops.n2_pid.d = 0
ops.n2_pid.amax = 100
ops.n2_pid.amin = 0
ops.n2_pid.alpha = 1
ops.n2_pid.beta = 1
ops.n2_pid.linearity = 1

In [18]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
    return f
def get_text(ops):

    text = \
"""
O2: P=%3d  I=%5d  Initial PV: %3d  k: %.4f/hr (%sx Est.)
N2: P=%3d  I=%4d  Set Point:  %3d  c: %s%%/hr

 """ % (ops.o2_pid.p, ops.o2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.n2_pid.p, ops.n2_pid.i, ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text
text = get_text(ops)

In [19]:
def run(ops):
    global data, data2, text
    data, data2 = do_sim(ops)
    text = get_text(ops)
    plot(ops)

In [20]:
seconds = 1
minutes = seconds * 60
hours = minutes * 60
days = hours * 24
ops.end = 48 * hours
sp = 40

ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 0

ops.n2_pid.p = -5
ops.n2_pid.i = 10
ops.n2_pid.amax = 100
ops.o2_pid.deadband = 0

ops.initial_pv = 100
ops.set_point = sp
ops.c = 0

# k_mult = 1.25
ops.k_mult = 1.1
ops.k = DOProcess.default_k
ops.initial_request_cno = (0.00, 0, 0.32)
ops.mode = 'm2a'
ops.plots.xscale = 'man'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 1
ops.delay = 0

run(ops)

NameError: name 'plot' is not defined

In [ ]:
ops.k*(100/20.95)*100

In [ ]:
# manually constructed list of [(o2p, o2i),(n2p, n2i)]
maybe_good_params = [
    [
        (10, 10),
        (-10, 100)
    ],
    [
        (2, 40),
        (-10, 50)
    ]
]

In [21]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a
    x, pv, mg, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a = list(zip(*data))
    uk, up, ui = list(zip(*data2))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / hours
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=sp, ls="--", color="black")
    # abs_err = np.abs(sp - pvs)
    # esq **= 2
    # esq = np.sqrt(esq)
    #ax1.plot(xs, abs_err, "red", ls="-", label="Err^2")


    #ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    #ax1.set_ylim((sp-20, sp+20))


    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, text, transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(640,720,50, 20))

Using matplotlib backend: TkAgg


In [22]:
plot(ops)

Parsing Data
Closing Plot
Plotting Data


In [23]:
class RingBuffer():
    """ Numpy-based ring buffer """

    def __init__(self, maxsize, dtype=np.float32):
        if not maxsize or maxsize < 0:
            raise ValueError("%s requires max size argument" % self.__class__.__name__)
        self._queue = np.zeros(maxsize, dtype)
        self._dtype = dtype
        self._maxsize = maxsize
        self._end = 0
        self._sz = 0

    def put(self, d):
        self._queue[self._end] = d
        self._end += 1
        if self._end == self._maxsize:
            self._end = 0
        if self._sz < self._maxsize:
            self._sz += 1

    def put_list(self, lst):
        """
        :param lst: list to extend data from
        :type lst: list | tuple | np.ndarray
        """
        slen = len(lst)
        if slen > self._maxsize:
            print(slen, self._maxsize, np.array(lst)[slen-self._maxsize:])
            self._queue = np.array(lst, dtype=self._dtype)[slen-self._maxsize:]
            self._sz = self._maxsize
            self._end = 0
            return
        if slen + self._end <= self._maxsize:
            start = self._end
            end = slen + self._end
            self._queue[start:end] = lst
            self._end = end
        else:
            # add slice in two steps
            first_step = self._maxsize - self._end
            self._queue[self._end: self._maxsize] = lst[:first_step]
            second_step = slen - first_step
            self._queue[:second_step] = lst[first_step:]
            self._end = second_step

        if self._sz < self._maxsize:
            self._sz += slen
        if self._end == self._maxsize:
            self._end = 0

    def extend(self, it):
        try:
            it.__len__  # list, tuple, nparray
        except AttributeError:
            it = tuple(it)
        self.put_list(it)

    def get(self):
        """
        Get method. Returns the entire queue but does NOT
        drain the queue.
        """
        if self._sz < self._maxsize:
            return self._queue[: self._end]
        else:
            return np.roll(self._queue, -self._end)

    def __len__(self):
        return self._sz

    def clear(self):
        self._end = 0
        self._sz = 0

In [24]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
# implement the default mpl key bindings
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import tkinter as tk
import queue


class Data():
    def __init__(self, pts):
        self.values = RingBuffer(pts)
        self.pending = []
        
    def put(self, v):
        self.pending.append(v)
        
    def push(self, n):
        self.values.extend(self.pending[:n])
        self.pending = self.pending[n:]
        
    def get(self):
        return self.values.get()

class SeriesData(Data):
    def __init__(self, name, ax, xdata, pts, **line_kw):
        self.name = name
        super().__init__(pts)
        self.pending = []
        self.x = xdata
        self.line, = ax.plot((), (), **line_kw)
        
    def update_line(self):
        self.line.set_data(self.x.get(), self.values.get())
        
    
# Plot classes contain custom configuration code
# so SimWindow doesn't need to know how to set them up

class Plot():
    def __init__(self, ax, xdata, pts, yformatter=None, *series_ops):
        self.ax = ax
        self.x = xdata
        self.series = []
        for ops in series_ops:
            name, kw = ops
            # default
            kw['ls'] = kw.get('ls') or "-"
            s = SeriesData(name, ax, xdata, pts, **kw)
            self.series.append(s)
            setattr(self, name, s)
        b = ax.get_position()
        ax.set_position([b.x0, b.y0, b.width*0.9, b.height])
        ax.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        ax.grid()
        if yformatter:
            ax.yaxis.set_major_formatter(yformatter)
            
    def put(self, *values):
        if len(values) != len(self.series):
            raise ValueError("Invalid argument to put: got %d values (expected %d)" \
                            % (len(values), len(self.series)))
        for s, v in zip(self.series, values):
            s.put(v)
            
    def push(self, n):
        for s in self.series:
            s.push(n)
            
    def update(self):
        for s in self.series:
            s.update_line()
        self.rescale()
        
    def rescale(self):
        self.ax.relim()
        self.ax.autoscale_view(True,True,True)
            

class PVPlot(Plot):
    """ Plot for PV """
    def __init__(self, ax, xdata, pts):
        series = [
            ("pv", dict(color="blue", ls="-", label="PV")),
        ]
        fm = FuncFormatter(lambda y, _: "%.2f%%"%y)
        super().__init__(ax, xdata, pts, fm, *series)
        
    def rescale(self):
        self.ax.relim()
        self.ax.autoscale_view(True,True,True)
        lower, upper = self.ax.get_ybound()
        padding = 5
        self.ax.set_ylim(lower - padding, upper + padding, True, None)
    
class GasesPlot(Plot):
    def __init__(self, ax, xdata, pts):
        
        series = [
            ("co2", dict(color="purple", ls="-", label="CO2")),
            ("n2", dict(color="red", ls="-", label="N2")),
            ("o2", dict(color="green", ls="-", label="O2")),
            ("air", dict(color="cyan", ls="-", label="Air")),
        ]
        fm = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))
        super().__init__(ax, xdata, pts, fm, *series)
        self.ax.set_ylim((0, 1.1))
        
    def rescale(self):
        self.ax.relim()
        self.ax.autoscale_view(True,True,False)
        
    
    
        
class PIDPlot(Plot):
    def __init__(self, ax, xdata, pts):
        series = [
            ("uk", dict(color="blue", ls="-", label="Uk")),
            ("up", dict(color="red", ls="-", label="Up")),
            ("ui", dict(color="green", ls="-", label="Ui")),
            ("ud", dict(color="purple", ls="-", label="Ud")),
        ]
        
        super().__init__(ax, xdata, pts, None, *series)
        




In [25]:
class SimConfig(OptionCategory):
    simops = ops or SimOps()
    xwindow_hrs = 20
    pause = False
    update_interval = 100
    time_factor = 3600
    time_unit = 3600
    max_iters = 3600 * 5  # max number of iters to simulate in one graph update interval
cfg = SimConfig()
print(cfg.jsonify())

{
    "time_factor": 3600,
    "simops": {
        "reactor_size": 80,
        "initial_actual_cno": [
            0.0004,
            0.7809,
            0.2095
        ],
        "c": 0,
        "initial_request_cno": [
            0.0,
            0,
            0.32
        ],
        "k_mult": 1.1,
        "set_point": 40,
        "delay": 0,
        "initial_pv": 100,
        "main_gas": 1.0,
        "o2_pid": {
            "d": 0,
            "beta": 1,
            "deadband": 0,
            "i": 10,
            "alpha": 0,
            "amax": 100,
            "linearity": 1,
            "p": 2,
            "amin": 0
        },
        "k": 0.1306,
        "reactor_volume": 55,
        "set_point_deadband": 1,
        "n2_pid": {
            "d": 0,
            "beta": 1,
            "deadband": 0,
            "i": 10,
            "alpha": 1,
            "amax": 100,
            "linearity": 1,
            "p": -5,
            "amin": 0
        },
        "mode": "m2a",
        

In [26]:

    


def do_sim_for_thread(self, stop_flag, pause_ack, resume_flag, resume_ack, xq, pvq, 
                      co2q, n2q, o2q, aq, ukq, upq, uiq, udq):
    ops = self.cfg.simops
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp-db, o2_req*100)
        n2_pid.man_to_auto(pv, sp+db, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(ops.k*ops.k_mult, ops.c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = cfg.time_unit
    # All queue references passed in are actually
    # wrappers around the actual queue containers
    # so the internal container can be manipulated
    # transparently from this function's point of view.
    xq_put = xq.put
    pvq_put = pvq.put
    co2q_put = co2q.put
    n2q_put = n2q.put
    o2q_put = o2q.put
    aq_put = aq.put
    ukq_put = ukq.put
    upq_put = upq.put
    uiq_put = uiq.put
    udq_put = udq.put
    
    values_this_update = 0
    max_per_update = self.time_factor * self.update_interval / 1000
    
    while True:
        # locks have high overhead for acquiring and releasing
        # so it would be a potential performance hit to including
        # a lock in the inner mainloop.
        # checking for boolean cfg.pause is atomic and fast,
        # so first check for pause boolean and then check the 
        # slower event synchronization flags to determine
        # course of action. 
            
        if cfg.pause:
            pause_ack.set()
            if stop_flag.is_set():
                break
            else:
                if not resume_flag.wait(5):
                    raise RuntimeError("Threading Deadlock")
                resume_ack.set()
        t += 1
        o2_req = o2_pid.step(pv, sp-db) / 100
        n2_req = n2_pid.step(pv, sp+db) / 100
        co2_req, o2_req, n2_req, air_req = ctrl.request(mg, co2_req, o2_req, n2_req)
        pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)
        
        uk = n2_pid.Uk
        up = n2_pid.Up
        ui = n2_pid.Ui
        ud = n2_pid.Ui
        
        xq_put(t/time_unit)
        
        pvq_put(pv)
        
        co2q_put(co2_req)
        n2q_put(n2_req)
        o2q_put(o2_req)
        aq_put(air_req)
        
        ukq_put(uk)
        upq_put(up)
        uiq_put(ui)
        udq_put(ud)

        values_this_update += 1


In [27]:
def do_sim_coroutine(self, xq, pvq, 
                      co2q, n2q, o2q, aq, ukq, upq, uiq, udq):
    ops = self.cfg.simops
    o2_pid = PIDController(pgain=ops.o2_pid.p,              itime=m2s(ops.o2_pid.i),
                            dtime=m2s(ops.o2_pid.d),        auto_max=ops.o2_pid.amax,
                            auto_min=ops.o2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.o2_pid.beta,
                            linearity=ops.o2_pid.linearity, alpha=ops.o2_pid.alpha,
                            deadband=ops.o2_pid.deadband)
    
    n2_pid = PIDController(pgain=ops.n2_pid.p,              itime=m2s(ops.n2_pid.i),
                            dtime=m2s(ops.n2_pid.d),        auto_max=ops.n2_pid.amax,
                            auto_min=ops.n2_pid.amin,       do_ifactor=True,
                            anti_windup_backcalc=True,      beta=ops.n2_pid.beta,
                            linearity=ops.n2_pid.linearity, alpha=ops.n2_pid.alpha,
                            deadband=ops.n2_pid.deadband)
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    if mode == "o2a":
        n2_req = 0
        o2_req = 0
        mode = "m2a"  # fallthrough below
    
    if mode == 'm2a':
        o2_pid.man_to_auto(pv, sp-db, o2_req*100)
        n2_pid.man_to_auto(pv, sp+db, n2_req*100)
    else:
        o2_pid.auto_to_auto(pv, o2_req*100)
        n2_pid.auto_to_auto(pv, o2_req*100)
        
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(ops.k*ops.k_mult, ops.c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = cfg.time_unit
    # All queue references passed in are actually
    # wrappers around the actual queue containers
    # so the internal container can be manipulated
    # transparently from this function's point of view.
    xq_put = xq.put
    pvq_put = pvq.put
    co2q_put = co2q.put
    n2q_put = n2q.put
    o2q_put = o2q.put
    aq_put = aq.put
    ukq_put = ukq.put
    upq_put = upq.put
    uiq_put = uiq.put
    udq_put = udq.put
    
    mi = cfg.max_iters
    yield
    while True:
        # locks have high overhead for acquiring and releasing
        # so it would be a potential performance hit to including
        # a lock in the inner mainloop.
        # checking for boolean cfg.pause is atomic and fast,
        # so first check for pause boolean and then check the 
        # slower event synchronization flags to determine
        # course of action. 
            
        iters = yield
        if iters < 0:
            continue
        elif iters > mi:
            iters = mi
            
        while iters > 0:
        
            t += 1
            iters -= 1
            o2_req = o2_pid.step(pv, sp-db) / 100
            n2_req = n2_pid.step(pv, sp+db) / 100
            co2_req, o2_req, n2_req, air_req = ctrl.request(mg, co2_req, o2_req, n2_req)
            pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)

            uk = n2_pid.Uk
            up = n2_pid.Up
            ui = n2_pid.Ui
            ud = n2_pid.Ui

            xq_put(t/time_unit)

            pvq_put(pv)

            co2q_put(co2_req)
            n2q_put(n2_req)
            o2q_put(o2_req)
            aq_put(air_req)

            ukq_put(uk)
            upq_put(up)
            uiq_put(ui)
            udq_put(ud)


In [28]:
import threading

class DOSimWindow():
    """ Sim Window handles common MPL functions & data storage 
    as well as non-MPL tkinter configuration 
    """
    def __init__(self, frame, sim_config):
        
        # Unpack config items
        self.cfg = sim_config
        self.update_interval = cfg.update_interval
        self.xwindow = cfg.xwindow_hrs
        
        # MPL objects
        self.fig = Figure()
        ax1 = self.fig.add_subplot(311)
        ax2 = self.fig.add_subplot(312)
        ax3 = self.fig.add_subplot(313)
        
        self.xdata = Data(self.xwindow*3600)
        self.pv_plot = PVPlot(ax1, self.xdata, self.xwindow*3600)
        self.gases_plot = GasesPlot(ax2, self.xdata, self.xwindow*3600)
        self.pid_plot = PIDPlot(ax3, self.xdata, self.xwindow*3600)

        # Tkinter setup
        self.root = frame
        
        self.fig_frame = tk.Frame(self.root)
        self.fig_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        # a tk.DrawingArea
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.fig_frame)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        self.toolbar = NavigationToolbar2TkAgg(self.canvas, self.root)
        self.toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        self._running = False
        self._monitor_id = None
        self._monitor_interval = 500
        self.root.protocol("WM_DELETE_WINDOW", self.destroy)
        
        def on_key_event(event):
            key_press_handler(event, self.canvas, self.toolbar)

        self.canvas.mpl_connect('key_press_event', on_key_event)
        
        # Threading support
        # self.sim_thread = None
        # self.pause_ack = threading.Event()
        # self.resume_flag = threading.Event()
        # self.resume_ack = threading.Event()
        # self.stop_flag = threading.Event()
        
        # coroutine support
        self.do_coro = None
        
        # Periodic update support
        self.updating = True
        
    def show(self):
        self._running = True
        self.begin_sim()
        self._monitor_running()
        self.schedule_update()
        self.root.mainloop()
        
        self.stop_flag.set()
        self.cfg.pause = True
        self.resume_flag.clear()
        self.resume_ack.clear()
        
    def begin_sim(self):
        self.resume_flag.clear()
        self.stop_flag.clear()
        self.cfg.pause = False
        self.sim_thread = threading.Thread(None, self._begin_sim, daemon=True)
        self.sim_thread.start()
        
    def begin_sim(self):
        self._begin_sim()
        
    def _begin_sim(self):
        do_sim_for_thread(self.cfg, self.stop_flag, self.pause_ack, self.resume_flag, self.resume_ack,
                         self.xdata, self.pv_plot.pv, self.gases_plot.co2,
                         self.gases_plot.n2, self.gases_plot.o2, self.gases_plot.air,
                         self.pid_plot.uk, self.pid_plot.up, self.pid_plot.ui,
                         self.pid_plot.ud)
        
    def _begin_sim(self):
        self.do_coro = do_sim_coroutine(self,
                         self.xdata, self.pv_plot.pv, self.gases_plot.co2,
                         self.gases_plot.n2, self.gases_plot.o2, self.gases_plot.air,
                         self.pid_plot.uk, self.pid_plot.up, self.pid_plot.ui,
                         self.pid_plot.ud)
        next(self.do_coro)
        
    def update(self):
        n = self.cfg.time_factor * self.update_interval / 1000
        n = int(n)
        self.do_coro.send(n)
        self.xdata.push(n)
        self.pv_plot.push(n)
        self.gases_plot.push(n)
        self.pid_plot.push(n)
        self.root.update()
        self.pv_plot.update()
        self.gases_plot.update()
        self.pid_plot.update()
        self.fig.canvas.draw()
        
#     def schedule_update(self):
#         def update(self=self):
#             self.cfg.pause = True
#             self.pause_ack.wait()
#             self.update()
#             self.cfg.pause = False
#             self.pause_ack.clear()
#             self.resume_flag.set()
#             if not self.resume_ack.wait(5):
#                 if self.sim_thread.is_alive():
#                     raise RuntimeError("Threading Deadlock")
#                 else:
#                     print("Thread dead, aborting update")
#             self.resume_flag.clear()
#             self.resume_ack.clear()
#             if self.updating:
#                 self.root.after(self.update_interval, update)
#         self.root.after(self.update_interval, update)
    def schedule_update(self):
        def update(self=self):
            self.update()
            if self.updating:
                self.root.after(self.update_interval, update)
        self.root.after(self.update_interval, update)    
    
    def _monitor_running(self):
        # hack to allow closing tkinter window by 
        # "X" button in this interactive prompt
        # unsure if this is an ipython/mpl artifact or not
        # but it works
        def monitor():
            if not self._running:
                self.root.quit()
                self.root.destroy()
            else:
                self.root.after(self._monitor_interval, monitor)
        self.root.after(self._monitor_interval, monitor)
        
    def destroy(self):
        self._running = False

In [29]:
cfg.update_interval = 50
cfg.time_factor = 3600
w = DOSimWindow(cfg)
w.show()

TypeError: __init__() missing 1 required positional argument: 'sim_config'

In [30]:
w.root.destroy()

NameError: name 'w' is not defined

In [ ]:
for plot in w.pv_plot, w.gases_plot, w.pid_plot:
    for s in plot.series:
        print(s.name, len(w.xdata.get()), len(s.get()))

In [ ]:
import cProfile
cProfile.run?

In [ ]:
# class PVPlot():
#     """ Plot for PV """
#     def __init__(self, ax, xdata, pts):
#         self.ax = ax
#         self.x = xdata
#         self.pv = SeriesData(ax, xdata, pts, "blue", ls="-", label="PV")
        
#     def add_data(self, t, pv):
#         self.pv.put(t)
        
#     def update(self):
#         self.pv.update_line()
    
    
# class GasesPlot():
#     def __init__(self, ax, xdata, pts):
#         self.x = xdata
#         self.ax = ax
        
#         self.n2 = SeriesData(ax, xdata, pts, "red", ls="-", label="N2")
#         self.o2 = SeriesData(ax, xdata, pts, "green", ls="-", label="O2")
#         self.air = SeriesData(ax, xdata, pts, "cyan", ls="-", label="Air")
#         self.co2 = SeriesData(ax, xdata, pts, "purple", ls="-", label="CO2")
        
#     def add_data(self, n, o, a):
#         self.n2.put(n)
#         self.o2.put(o)
#         self.air.put(a)
        
    
    
        
# class PIDPlot():
#     def __init__(self, ax, xdata, pts):
#         self.x = xdata
#         self.ax = ax
#         self.uk = SeriesData(ax, pts, "blue", ls="-", label="Uk")
#         self.up = SeriesData(ax, pts, "red", ls="-", label="Up")
#         self.ui = SeriesData(ax, pts, "green", ls="-", label="Ui")
#         self.ud = SeriesData(ax, pts, "purple", ls="-", label="Ud")

In [ ]:
def paste(cells, data, offset=0, headers=None):
    headers = headers or [("T", "PV", "OP")]
    ws = cells.Parent
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, len(headers)+offset)).Value = headers
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        bottomleft = topleft.Offset(len(data), 1)
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data
        chart = ws.ChartObjects(1).Chart
        s1 = chart.SeriesCollection(1)
        s1.XValues = cells.Range(topleft, bottomleft)
        s1.Values = cells.Range(topleft.Offset(1,2), bottomleft.Offset(1,2))
        s1.Name = cells(1,1).Offset(1, 2).Value
        
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [ ]:
# def format_chart(ws, cells, topleft, bottomleft, 
#                  chart, gases, offset2, scnd=False):
#     for i, g in enumerate(gases):
#         try:
#             s = chart.SeriesCollection(i+2)
#         except com_error:
#             s = chart.SeriesCollection().NewSeries()
#         s.XValues = cells.Range(topleft, bottomleft)
#         ytop = topleft.Offset(1, i+offset2)
#         ybot = bottomleft.Offset(1, i+offset2)
#         yrng = cells.Range(ytop, ybot)
#         yrng.NumberFormat = "0.00%"
#         s.Values = yrng
#         s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+4).Address
#         if scnd:
#             s.AxisGroup = xlc.xlSecondary
            

def paste2(cells, data, offset=0, headers=None):
    headers = headers or [("T", "PV", "OP")]
    ws = cells.Parent
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, len(headers)+offset)).Value = headers
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        bottomleft = topleft.Offset(len(data), 1)
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data

        # pv
        try:
            chart = ws.ChartObjects(1).Chart
        except com_error:
            chart = CreateChart(ws, Left=1, Top=1, Width=500, Height=220)
        try:
            s1 = chart.SeriesCollection(1)
        except com_error:
            s1 = chart.SeriesCollection().NewSeries()
        chart = ws.ChartObjects(1).Chart
        chart.HasTitle=True
        chart.ChartTitle.Text = "DOPV(%)"
        s1 = chart.SeriesCollection(1)
        s1.XValues = cells.Range(topleft, bottomleft)
        ytop = topleft.Offset(1,2)
        ybot = bottomleft.Offset(1,2)
        s1.Values = cells.Range(ytop, ybot)
        s1.Name = "=" + "'%s'!"%ws.Name+ cells(1,2).Address
        
        # gas flows
        gases = "CO2 N2 O2".split()
        for i, g in enumerate(gases):
            try:
                s = chart.SeriesCollection(i+2)
            except com_error:
                s = chart.SeriesCollection().NewSeries()
            s.XValues = cells.Range(topleft, bottomleft)
            ytop = topleft.Offset(1, i+4)
            ybot = bottomleft.Offset(1, i+4)
            yrng = cells.Range(ytop, ybot)
            yrng.NumberFormat = "0.00%"
            s.Values = yrng
            s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+4).Address
        # for some reason, need to loop again for this
        for i in range(2, chart.SeriesCollection().Count+1):
            s1 = chart.SeriesCollection(i)
            s1.AxisGroup = xlc.xlSecondary
            s1.AxisGroup = xlc.xlSecondary
        
        # second chart, actual O2 concentrations
        try:
            chart2 = ws.ChartObjects(2).Chart
            delete=False
            start = 1
        except com_error:
            t = chart.Parent.Top
            h = chart.Parent.Height
            chart2 = CreateChart(ws, Left=1, Top=t + h + 20, Width=500)
            chart.ChartType = xlc.xlXYScatter
            s1 = chart2.SeriesCollection().NewSeries()
            delete = True
            start = 2
        for i, g in enumerate(gases, start):
            try:
                s = chart2.SeriesCollection(i)
            except com_error:
                s = chart2.SeriesCollection().NewSeries()
            s.XValues = cells.Range(topleft, bottomleft)
            ytop = topleft.Offset(1, i+7-start)
            ybot = bottomleft.Offset(1, i+7-start)
            yrng = cells.Range(ytop, ybot)
            yrng.NumberFormat = "0.00%"
            s.Values = yrng
            s.Name = "=" + "'%s'!"%ws.Name+ cells(1, i+7-start).Address
        if delete:
            s1.Delete()
        
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [ ]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
xl = Excel()

wb = xl.ActiveWorkbook
wb = wb or xl.Workbooks.Add()

ws = xl.ActiveSheet
ws = ws or wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [ ]:
ops.end=100000
ops.o2_pid.p = 1
ops.o2_pid.i = 0.1
ops.set_point = 50
ops.n2_pid.p = 0
ops.n2_pid.i = 0
ops.n2_pid.amax = 0
ops.c = 0.002
step = 50
with screen_lock(xl):
    clear(cells, data[::step], 0)
    data = do_sim(ops)
    headers = "Time(sec), DOPV(%), Main Gas, CO2 Req, N2 Req, " \
                "O2 Req, CO2 Headspace, N2 Headspace, O2 Headspace"
    headers = headers.split(", ")
    paste2(cells, data[::step], 0, headers)

In [ ]:
xl=Excel()
wb = xl.RecentFiles(1).Open()

In [ ]:
n2data = xl.Selection.Value2